In [1]:
import pandas as pd
import csv
import pickle
import os
import numpy as np
import json
%load_ext autoreload
%autoreload 2

In [2]:
# read the pre-processed data
with open("data/twitter-swisscom/pId_geo_df_total.pickle", "rb") as h:
    pId_geo_df_total = pickle.load(h)
with open("data/twitter-swisscom/create_twitter_total.pickle", "rb") as h:
    create_twitter_total = pickle.load(h)
with open("data/twitter-swisscom/user_info_total.pickle", "rb") as h:
    user_info_total = pickle.load(h)
with open("data/twitter-swisscom/geo_info_total.pickle", "rb") as h:
    geo_info_total = pickle.load(h)

**pId_geo_df_total** contains the geo information of the every *placeId* appeared in the twitter data

In [3]:
pId_geo_df_total.head()

,geo,country_code,state
placeId,,,
000831c517105356,"{'osm_id': '257340988', 'lat': '17.102536', 'p...",th,จังหวัดสกลนคร
000a93ad12003aaa,"{'lon': '7.5100197', 'lat': '46.8912965', 'osm...",ch,Bern - Berne
000b5d1aada9dcaa,"{'lon': '7.44578927095059', 'lat': '46.9525886...",ch,Bern - Berne
0010c7694b04e371,"{'lon': '8.50580530903368', 'lat': '47.4261841...",ch,Zürich
0013241b7342de79,"{'lon': '8.3072505', 'lat': '47.0461309', 'osm...",ch,Luzern


List of countries that those *placeId* belongs to.

In [4]:
pId_geo_df_total['country_code'].unique()

array(['th', 'ch', 'us', 'ae', 'it', 'fr', 'gh', 'do', 'de', 'cz', 'kw',
       'au', 'dz', 've', 'sa', 'ng', 'tr', 'fi', 'nz', 'kr', 'cl', 'mx',
       'co', 'vn', 'cn', 'at', 'uy', 'nl', 'be', 'ua', 'gb', 'es', 'cf',
       'br', 'cd', 'ru', 'za', 'ca', 'ar', 'pt', 'kz', 'id', 'pl', 'se',
       'dk', 'na', 'no'], dtype=object)

**create_twitter_total** contains the informations of the dates each user post their twitter.

In [5]:
create_twitter_total.head()

1,userId,createdAt,placeId,weekday,day,hour,month
0,30445147,2013-04-24 01:42:07,508f024bc856afc5,2,24,1,4
1,1136150214,2013-04-24 01:42:39,d969dab4efecff93,2,24,1,4
2,853308949,2013-04-24 01:42:44,a573c842499b9471,2,24,1,4
3,1136150214,2013-04-24 01:43:01,d969dab4efecff93,2,24,1,4
4,1136150214,2013-04-24 01:44:13,d969dab4efecff93,2,24,1,4


In [6]:
# add a column for number of year
create_twitter_total_flat = create_twitter_total.reset_index()
createdAt_list = create_twitter_total_flat['createdAt']
year_list = createdAt_list.apply(lambda x: x[0:4])
create_twitter_total_flat['year'] = year_list

In [7]:
create_twitter_total_flat.head()

1,index,userId,createdAt,placeId,weekday,day,hour,month,year
0,0,30445147,2013-04-24 01:42:07,508f024bc856afc5,2,24,1,4,2013
1,1,1136150214,2013-04-24 01:42:39,d969dab4efecff93,2,24,1,4,2013
2,2,853308949,2013-04-24 01:42:44,a573c842499b9471,2,24,1,4,2013
3,3,1136150214,2013-04-24 01:43:01,d969dab4efecff93,2,24,1,4,2013
4,4,1136150214,2013-04-24 01:44:13,d969dab4efecff93,2,24,1,4,2013


Since we observe that there are twitters at 1994 (it's impossible since Twitter was not born at that moment), we decide to remove all the twitter that posted before 2009. The reason to choose 2009 is the number of twitters before 2009 is very few, around 40.

In [8]:
create_twitter_total_flat_after_2009 = create_twitter_total_flat[create_twitter_total_flat['year']>'2009']

We count the number of twitters for given date (use day as unit) and given placeId.

In [9]:
transed = create_twitter_total_flat_after_2009.groupby(['placeId','year','month','day']).count()['userId']

In [10]:
transed.name = 'twittwer_count'
transed2 = transed.reset_index()
transed2.head()

,placeId,year,month,day,twittwer_count
0,000831c517105356,2014,12,26,1
1,000a93ad12003aaa,2010,10,4,1
2,000a93ad12003aaa,2010,11,28,1
3,000a93ad12003aaa,2010,12,7,1
4,000a93ad12003aaa,2011,1,26,1


We join the above Dataframe with *pId_geo_df_total*.

In [11]:
transed3 = transed2.join(pId_geo_df_total[['country_code', 'state']], on='placeId')
# for those which cannot find its geolocation, we assign 'unspecified' to its state name. 
# We remove these placeId and corresponding dates and twittwer_count
transed3 = transed3[transed3.state != 'unspecified']
# We pivote transform this table to be a multi-layer dataframe.
aimed_table = transed3.pivot_table(columns=['year','month','day'],
                                   index =['country_code', 'state'], values='twittwer_count', 
                                   fill_value = 0, aggfunc= 'sum')

In [12]:
aimed_table

year                                                            2010           \
month                                                              3            
day                                                               9  10 11 12   
country_code state                                                              
ae           أبو ظبي                                               0  0  0  0   
ar           CHB                                                   0  0  0  0   
             Mza.                                                  0  0  0  0   
at           Niederösterreich                                      0  0  0  0   
             Salzburg                                              0  0  0  0   
             Tirol                                                 0  0  0  0   
             Vorarlberg                                            0  0  0  0   
au           Australian Capital Territory                          0  0  0  0   
             NSW                                                   0  0  0  0   
             Victoria                                              0  0  0  0   
             Western Australia                                     0  0  0  0   
be           Région de Bruxelles-Capitale - Brussels Hoofdst...    0  0  0  0   
             Vlaanderen                                            0  0  0  0   
br           BA                                                    0  0  0  0   
             RJ                                                    0  0  0  0   
             SC                                                    0  0  0  0   
             SP                                                    0  0  0  0   
ca           British Columbia                                      0  0  0  0   
             Ontario                                               0  0  0  0   
             Québec                                                0  0  0  0   
             Saskatchewan                                          0  0  0  0   
cd           Tshopo                                                0  0  0  0   
cf           Ouaka                                                 0  0  0  0   
ch           Aargau                                                0  0  0  0   
             Appenzell Ausserrhoden                                0  0  0  0   
             Appenzell Innerrhoden                                 0  0  0  0   
             Basel-Landschaft                                      1  0  0  0   
             Basel-Stadt                                           0  0  1  0   
             Bern - Berne                                          0  0  1  0   
             Fribourg - Freiburg                                   0  0  0  0   
...                                                              ... .. .. ..   
us           Alaska                                                0  0  0  0   
             California                                            0  0  0  0   
             Florida                                               0  0  0  0   
             Hawaii                                                0  0  0  0   
             Illinois                                              0  0  0  0   
             Kansas                                                0  0  0  0   
             Louisiana                                             0  0  0  0   
             Maryland                                              0  0  0  0   
             Massachusetts                                         0  0  0  0   
             Michigan                                              0  0  0  0   
             Mississippi                                           0  0  0  0   
             Missouri                                              0  0  0  0   
             Nevada                                                0  0  0  0   
             New Jersey                                            0  0  0  0   
             New Mexico        

To make it easier to read the data in Javascript, we store above dataframe into two dictionaries, one called *place_dict* and the other *data_dict*.

*place_dict* is a nested dictionary, the key is the country name, the value is another dictionary whose key is state name and value is a list of number of twitters each day.

*data_dict* is dictionary whose key is sequence number and the value is the list of year, month and day.

Above two dictionaries will be fed in the Javescropt to compute **lead_json.html**

In [13]:
place_dict = {}
for i in aimed_table.index.levels[0]:
    inter_dic = {}
    for j in aimed_table.loc[i].index:
        inter_dic[j] = ((aimed_table.loc[(i,j)].values).tolist())
    place_dict[i] = inter_dic

In [15]:
with open("webpage/place_dict2.json", "w") as outfile:
    json.dump(place_dict, outfile)

In [16]:
date_list = aimed_table.columns.values
date_list
data_dict = {}
for i in range(len(date_list)):
    data_dict[i] = list(date_list[i])

In [17]:
with open("webpage/date_dict2.json", "w") as outfile:
    json.dump(data_dict, outfile)

In [18]:
data_dict

{0: ['2010', 3, 9],
 1: ['2010', 3, 10],
 2: ['2010', 3, 11],
 3: ['2010', 3, 12],
 4: ['2010', 3, 13],
 5: ['2010', 3, 14],
 6: ['2010', 3, 15],
 7: ['2010', 3, 16],
 8: ['2010', 3, 17],
 9: ['2010', 3, 18],
 10: ['2010', 3, 19],
 11: ['2010', 3, 20],
 12: ['2010', 3, 21],
 13: ['2010', 3, 22],
 14: ['2010', 3, 23],
 15: ['2010', 3, 24],
 16: ['2010', 3, 25],
 17: ['2010', 3, 26],
 18: ['2010', 3, 27],
 19: ['2010', 3, 28],
 20: ['2010', 3, 29],
 21: ['2010', 3, 30],
 22: ['2010', 3, 31],
 23: ['2010', 4, 1],
 24: ['2010', 4, 2],
 25: ['2010', 4, 3],
 26: ['2010', 4, 4],
 27: ['2010', 4, 5],
 28: ['2010', 4, 6],
 29: ['2010', 4, 7],
 30: ['2010', 4, 8],
 31: ['2010', 4, 9],
 32: ['2010', 4, 10],
 33: ['2010', 4, 11],
 34: ['2010', 4, 12],
 35: ['2010', 4, 13],
 36: ['2010', 4, 14],
 37: ['2010', 4, 15],
 38: ['2010', 4, 16],
 39: ['2010', 4, 17],
 40: ['2010', 4, 18],
 41: ['2010', 4, 19],
 42: ['2010', 4, 20],
 43: ['2010', 4, 21],
 44: ['2010', 4, 22],
 45: ['2010', 4, 23],
 46: ['2